In [13]:
# libraries
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Model
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Concatenate


In [2]:
labels = ['alert', 'slight_drowsy','mod_drowsy','very_drowsy'] # alert, slightly drowsy, moderately drowsy and very drowsy

In [3]:
# Load the CSV file into a DataFrame
df = pd.read_csv('data/features/data_features.csv')

# Perform one-hot encoding using get_dummies()
encoded_columns = pd.get_dummies(df['target'])

# Reorder the columns of encoded_columns to match the desired order
encoded_columns = encoded_columns[labels]

# Concatenate the one-hot encoded columns with the original DataFrame
df_encoded = pd.concat([df, encoded_columns], axis=1)

df_encoded.head()

,blinks,right_EAR,left_EAR,blink_duration,blink_frequency,MAR,image,target,alert,slight_drowsy,mod_drowsy,very_drowsy
0,0,0.390506,0.389952,0.057747,13.800623,0.126324,alert_1_0.png,alert,1,0,0,0
1,0,0.447508,0.417273,0.057747,13.800623,0.067651,alert_1_1.png,alert,1,0,0,0
2,0,0.448586,0.430567,0.057747,13.800623,0.063540,alert_1_2.png,alert,1,0,0,0
3,0,0.443849,0.409959,0.057747,13.800623,0.058703,alert_1_3.png,alert,1,0,0,0
4,0,0.445516,0.409511,0.057747,13.800623,0.057816,alert_1_4.png,alert,1,0,0,0


In [4]:
# Split the DataFrame into train, test, and validation sets
train_df, test_df = train_test_split(df_encoded, test_size=0.3, random_state=1)
test_df, val_df = train_test_split(test_df, test_size=0.5, random_state=1)

In [5]:
# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Select the columns you want to scale
columns_to_scale = ["right_EAR", "left_EAR", "blink_duration", "blink_frequency", "MAR"]
# Scale the selected columns
train_df[columns_to_scale] = scaler.fit_transform(train_df[columns_to_scale])
test_df[columns_to_scale] = scaler.transform(test_df[columns_to_scale])
val_df[columns_to_scale] = scaler.transform(val_df[columns_to_scale])

C:\Users\carlo\AppData\Local\Temp\ipykernel_9248\3985704919.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df[columns_to_scale] = scaler.fit_transform(train_df[columns_to_scale])


In [6]:
# Save the train, test, and validation DataFrames to separate CSV files if desired
train_df.to_csv('data/features/train_features.csv', index=False)
test_df.to_csv('data/features/test_features.csv', index=False)
val_df.to_csv('data/features/val_features.csv', index=False)

In [8]:
# Create ImageDataGenerators for train, test, and validation sets
train_datagen = ImageDataGenerator(rescale=1.0 / 255)  # Normalize pixel values
test_datagen = ImageDataGenerator(rescale=1.0 / 255)  # Normalize pixel values
val_datagen = ImageDataGenerator(rescale=1.0 / 255)  # Normalize pixel values

# Create train, test, and validation image generators using flow_from_dataframe
train_generator = train_datagen.flow_from_dataframe(
    dataframe=train_df,
    directory='data/images/',
    x_col='image',
    y_col=labels,
    batch_size=32,
    target_size=(720, 1280),
    class_mode='raw',
    shuffle=False, # the train-test split already shuffled so no need to do it again
    seed=1
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory='data/images/',
    x_col='image',
    y_col=labels,
    batch_size=32,
    target_size=(720, 1280),
    class_mode='raw',
    shuffle=False,
    seed=1
)

val_generator = val_datagen.flow_from_dataframe(
    dataframe=val_df,
    directory='data/images/',
    x_col='image',
    y_col=labels,
    batch_size=32,
    target_size=(720, 1280),
    class_mode='raw',
    shuffle=False,
    seed=1
)


Found 6300 validated image filenames.
Found 1350 validated image filenames.
Found 1350 validated image filenames.


In [9]:
X,y = train_generator.next()
print(X.shape, y.shape)

(32, 720, 1280, 3) (32, 4)


In [10]:
train_df.drop(columns=['image','target']+labels).columns.tolist()

['blinks', 'right_EAR', 'left_EAR', 'blink_duration', 'blink_frequency', 'MAR']

In [11]:
# Custom generator function to yield both image and feature data
def custom_generator(image_generator, feature_data, batch_size):
    feature_index = 0
    while True:
        batch_images, batch_labels = next(image_generator)
        batch_features = feature_data[feature_index : feature_index + batch_size]
        feature_index += batch_size
        yield [batch_images, batch_features], batch_labels

# Create custom generators combining image and feature data
train_generator_with_features = custom_generator(train_generator, train_df.drop(columns=['image','target']+labels), batch_size=32)
test_generator_with_features = custom_generator(test_generator, test_df.drop(columns=['image','target']+labels), batch_size=32)
val_generator_with_features = custom_generator(val_generator, val_df.drop(columns=['image','target']+labels), batch_size=32)

In [12]:
[x1,x2],y=next(train_generator_with_features)
print(x1.shape, x2.shape, y.shape)

(32, 720, 1280, 3) (32, 6) (32, 4)


In [14]:
# Define the image input branch
image_input = Input(shape=(720, 1280, 3))
conv1 = Conv2D(32, (3, 3), activation='relu')(image_input)
pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
# Add more convolutional and pooling layers as needed

# Define the numerical feature input branch
feature_input = Input(shape=(6,))
dense1 = Dense(64, activation='relu')(feature_input)
# Add more dense layers as needed

# Concatenate the outputs from both branches
concatenated = Concatenate()([Flatten()(pool1), dense1])

# Add a final dense layer for prediction
output = Dense(4, activation='softmax')(concatenated)

# Create the model
model = Model(inputs=[image_input, feature_input], outputs=output)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [15]:
# fit network
model.fit(train_generator_with_features, epochs=20, verbose=2, validation_data=val_generator_with_features)

Epoch 1/20
